# Taxi Trajectory Prediction

This notebook is part of the Coursera, IBM Advanced Data Science Capstone Project
Cerated by Engin Cicek, ecicek@gmail.com

Data used in this Capstone Project is found on kaggle website titled as "ECML/PKDD 15: Taxi Trajectory Prediction (I)" 

Dataset can be found at : https://www.kaggle.com/c/pkdd-15-predict-taxi-service-trajectory-i

---------------------------------------------------------------------------------------------------------

Cehecklist for Week2:

- Created an ETL Notebook

- Added Data Cleansing code to the ETL Notebook

- Created a Feature Creation Notebook

- Transformed Features

- Created additional Features

---------------------------------------------------------------------------------------------------------------

ETL :

First stage of is for the "Extract Transform Load (ETL)" stage of the Process Model used for this data science project.

Deliverable of this notebook is the data to be used in other stages of the process.


In [1]:
import pandas as pd
import numpy as np

Getting Data

In [2]:
df = pd.read_csv('train.csv')

---------------------------------------------------------------------------------------------------------------------

DATA CLEANSING :


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1710670 entries, 0 to 1710669
Data columns (total 9 columns):
TRIP_ID         int64
CALL_TYPE       object
ORIGIN_CALL     float64
ORIGIN_STAND    float64
TAXI_ID         int64
TIMESTAMP       int64
DAY_TYPE        object
MISSING_DATA    bool
POLYLINE        object
dtypes: bool(1), float64(2), int64(3), object(3)
memory usage: 106.0+ MB


There are 1,710,670 rows and 9 columns in data.

In [4]:
df.sample(10)

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
788360,1387279893620000398,B,NaN,27.0,20000398,1387279893,A,False,"[[-8.608725,41.147721],[-8.608644,41.147685],[..."
782447,1387148341620000423,A,5358.0,NaN,20000423,1387148341,A,False,"[[-8.60832,41.152905],[-8.608311,41.152707],[-..."
511708,1382119581620000258,A,58108.0,NaN,20000258,1382119581,A,False,"[[-8.638479,41.162328],[-8.638362,41.162283],[..."
931460,1389982794620000267,B,NaN,35.0,20000267,1389982794,A,False,"[[-8.64954,41.167116],[-8.649612,41.167179],[-..."
42580,1373384145620000468,B,NaN,9.0,20000468,1373384145,A,False,"[[-8.606403,41.144706],[-8.606574,41.144733],[..."
270944,1377990487620000179,B,NaN,56.0,20000179,1377990487,A,False,"[[-8.591229,41.162706],[-8.59104,41.162463],[-..."
166745,1375726902620000361,B,NaN,12.0,20000361,1375726902,A,False,"[[-8.630712,41.154957],[-8.630712,41.154957],[..."
1388082,1398776250620000640,B,NaN,14.0,20000640,1398776250,A,False,"[[-8.610876,41.148909],[-8.611074,41.14908],[-..."
593705,1383592936620000174,A,2002.0,NaN,20000174,1383592936,A,False,"[[-8.638659,41.159079],[-8.638569,41.159079],[..."
522642,1382369466620000521,B,NaN,27.0,20000521,1382369466,A,False,"[[-8.608725,41.147811],[-8.608716,41.147793],[..."


TRIP_ID is the unique identifier (primary key) for this table. It is actually the combination of TIMESTAMP and TAXI ID columns for the related row (It is one of the ways used for generating row identifiers).

I will go through each other columns one by one in UNIVARIATE ANALYSIS below for better understanding of the data and for checking whether any cleaning is needed for data validity and integrity. 

----------------------------------------------------------------------------------------------------------

UNIVARIATE ANALYSIS

----------------------------------------------------------------------------------------------------------

CALL_TYPE Column :

Description of the Column(as given in Data Desciption at the web address of data) :

It identifies the way used to demand this service. It may contain one of three possible values:

- ‘A’ if this trip was dispatched from the central;

- ‘B’ if this trip was demanded directly to a taxi driver on a specific stand;

- ‘C’ otherwise (i.e. a trip demanded on a random street).


In [5]:
df['CALL_TYPE'].value_counts(ascending=False)

B    817881
C    528019
A    364770
Name: CALL_TYPE, dtype: int64

In [6]:
df['CALL_TYPE'].isnull().sum()

0

CALL_TYPE column is filled with Categorical Nominal Values

The is no null value.

The most common is the Stand_Based calls, followed by Random-street calls as the second most common and last one is the taxi-cnetral based calls.

------------------------------------------------------------------------------------------------------

ORIGIN_CALL Column :

Description of the Column:

It contains an unique identifier for each phone number which was used to demand, at least, one service.

It identifies the trip’s customer if CALL_TYPE=’A’. Otherwise, it assumes a NULL value;

In [7]:
len(df['ORIGIN_CALL'].unique())-1 #minus 1 is for null values

57105

These are 57,105 unique customers that used taxi-central for calling a taxi.


In [8]:
df['ORIGIN_CALL'].isnull().sum()

1345900

There are 1,345,900 null values in this column but these do not have to be missing values. Because, only trips with taxi station calls can have customer ids (phone numbers). 

In order to verify whether this column has values consistently as it is described so, I will alseo run 2 checks:

- Check-1: There should not be any rows with null value in ORIGIN_CALL column if the trip is recorded as type 'A' Call

- Check-2: There should not be any rows with non-null value in ORIGIN_CALL column if the trip is recorded as type 'B' or 'C' Call_Type

In [9]:
check1=df.loc[(df['ORIGIN_CALL'].notnull())& (df['CALL_TYPE']!='A'),'TRIP_ID']

In [10]:
check1

Series([], Name: TRIP_ID, dtype: int64)

In [11]:
check2=df.loc[(df['ORIGIN_CALL'].isnull())& (df['CALL_TYPE']=='A'),'TRIP_ID']

In [12]:
check2

Series([], Name: TRIP_ID, dtype: int64)

Below is another way of checking both. Number of type-A call types + number of null values in ORIGIN_CALL columns shoul be equal to total number of rows.

In [13]:
len(df[df.loc[:,'CALL_TYPE']=='A'])+df['ORIGIN_CALL'].isnull().sum()-len(df)

0

Checks show that there is not any inconsistency about what the ORIGIN_CALL tells us.

Values are float but obviously there is no need to check for outliers. 

We don't have any other information (like age, gender etc.) about the customers other than their phone number within this identifier.

----------------------------------------------------------------------------------------------------------------------

We now know that there are 57,105 unique customers used taxi-central for calling taxi for their trips.
We also know that there are 364,770 trips started with calls made with the taxi_central (from CALL_TYPE calumn's category counts)

Then, below is the avarage number of calls a customer made to the tavi-central (for only those using taxi-central)

In [14]:
len(df[df.loc[:,'CALL_TYPE']=='A'])/(len(df['ORIGIN_CALL'].unique())-1)

6.387706855791962

And, below is the maximum number of calls received from a customer.

In [15]:
np.max(df['ORIGIN_CALL'].value_counts())

57571

In [16]:
np.max(df['ORIGIN_CALL'].value_counts())/len(df[df.loc[:,'CALL_TYPE']=='A'])

0.15782822052252105

It looks that almost 16% of the calls to the taxi-central are made by one customer.

This customer may be a firm, airport or big hotel. But, still this amount of calls from one location does not really make sense, since that would make aproximately 158 calls/ day and 6 calls/ hour in average (it is a 365-day data).

Looking at these averages, this calumn does not look reliable for futher analysis and model building.

It is better consider droping this column from the data that will be used for further stages and training model.

For now I will keep it and give my final decision after looking at its correlation with trip distance etc.

At this step, I still don't have the starting_locations of the trips. They need to be extracted from POLYLINE column that contains all location information of the total trip. I will come back to this point after I generate new columns (starting_location, end_location of the trips) from POLYLINE column. Then I will do one more round of checking for data cleansing needs.

------------------------------------------------------------------------------------------------------

ORIGIN_STAND :

Description of the Column:

It contains an unique identifier for each phone number which was used to demand, at least, one service.

It identifies the trip’s customer if CALL_TYPE=’A’. Otherwise, it assumes a NULL value.

In [17]:
len(df['ORIGIN_STAND'].unique())-1

63

These are 63 unique taxi stands.

In [18]:
df['ORIGIN_STAND'].isnull().sum()

904091

There are 904,084 null values in this column but these do not have to be missing values. Because, only trips from taxi stands can have taxistand IDs. 

In order to verify whether this column has values as it is described so, I will run 2 checks:

- There should not be any rows with null value in ORIGIN_STAND column if the trip is recorded as type 'B' Call

- There should not be any rows with non-null value in ORIGIN_STAND column if the trip is recorded as type 'A' or 'C' Call_Type

In [19]:
check3=df.loc[(df['ORIGIN_STAND'].notnull())& (df['CALL_TYPE']!='B'),'TRIP_ID']

In [20]:
check3

Series([], Name: TRIP_ID, dtype: int64)

In [21]:
check4=df.loc[(df['ORIGIN_STAND'].isnull())& (df['CALL_TYPE']=='B'),'TRIP_ID']

In [22]:
len(check4)

11302

In [23]:
#Another way of doing Check4
len(df[df.loc[:,'CALL_TYPE']=='B'])+df['ORIGIN_STAND'].isnull().sum()-len(df)

11302

First check has passed and show that there is not any row with Origin_Stand value is null and Call_Type is 'B' (which is consistency with what the ORIGIN_CALL column is supposed to give)

However, the second check has not passed and tells us that there are non-null values is ORIGIN_STAND calumn even the trip is not recorded as 'B' as CALL_TYPE.

This may be the result of :

- either misrecording CALL_TYPE as A or C instead of B

- or misrecording ORIGIN_STAND as some value (taxi stand id) instead of recording it as NULL.

In [24]:
len(check4)/len(df)

0.006606768108401971

These rows that are inconsistent in terms of CALL_TYPE and ORIGIN_STAND make up 0.6% of the total data which can be considered very few and can be cancelled out from the data.

However, I will check if these rows are actually belong to taxi stand calls (Type B) by cross-checking the the starting_point of the trip with the location of the taxi stand. I can do this by looking at another row that is known to be a taxi stand with the same ID.

At this step, I still don't have the starting_locations of the trips. They need to be extracted from POLYLINE column that contains all location information of the total trip. I will come back to this point after I generate new columns (starting_location, end_location of the trips) from POLYLINE column.

_______________________________________________________________________________________________________________________

TAXI ID :

Description of the Column:

It contains an unique identifier for the taxi driver that performed each trip.

In [25]:
len(df['TAXI_ID'].unique())

448

In [26]:
df['TAXI_ID'].isnull().sum()

0

There are 448 unique taxi drivers within the data. No missing value.

I don't think that the taxi driver would have any effect on taxi_call times, customer's destinations etc.

However, travelled distance between two same start and end locations may differ depending on the taxi driver (that happens in some cities). 

Therefore, this column can be used in testing this hyphotesis during further analysis and model building steps.

I will set forward my questions and related hyphotesis afterwards (Effect of taxi driver on travel distance and time is one of my potential questions).

------------------------------------------------------------------------------------------------------------------

DAY_TYPE :

Description of the Column:

It identifies the daytype of the trip’s start. It assumes one of three possible values:

- 'B’ if this trip started on a holiday or any other special day (i.e. extending holidays, floating holidays, etc.);
- ‘C’ if the trip started on a day before a type-B day;
- ‘A’ otherwise (i.e. a normal day, workday or weekend).

In [27]:
df['DAY_TYPE'].value_counts()

A    1710670
Name: DAY_TYPE, dtype: int64

In [28]:
df['DAY_TYPE'].isnull().sum()

0

It looks that this row has only A-type Days and no missing value. This is no variation in column values and this does not make any contribution to any analysis and model to be built. Therefore, I will just drop this column !!!

In [29]:
df.drop('DAY_TYPE', axis=1, inplace=True)

MISSING_DATA :

Description of the Column:

It is FALSE when the GPS data stream is complete and TRUE whenever one (or more) locations are missing

In [30]:
df['MISSING_DATA'].value_counts()

False    1710660
True          10
Name: MISSING_DATA, dtype: int64

There are very few non-null value in this column and it is impossible to find which GPS coordinate is missing during the trip. Therefore, I will just cancel out these 10 rows from the data and drop this column since there will be no variation within this column !!! 

In [31]:
df=df.loc[df['MISSING_DATA']==False,:]

In [32]:
df.drop('MISSING_DATA', axis=1, inplace=True)

-------------------------------------------------------------------------------------------------------------

TIMESTAMP :

Description of the Column: Unix Timestamp (in seconds). It identifies the trip’s start.

In [33]:
df['TIMESTAMP'].isnull().sum()

0

There is no NULL value in any row of this Column.

I find it more useful to add a new column in datetime type format and drop TIMESTAMP column extract month, day of the week, hour of the day from this new column for further analysis and model building steps. I will do this in the next step which will be on Feature Engineering.

--------------------------------------------------------------------------------------------------------------------

POLYLINE Column :

It contains a list of GPS coordinates (i.e. WGS84 format) mapped as a string. The beginning and the end of the string are identified with brackets (i.e. [ and ], respectively). Each pair of coordinates is also identified by the same brackets as [LONGITUDE, LATITUDE]. This list contains one pair of coordinates for each 15 seconds of trip. The last list item corresponds to the trip’s destination while the first one represents its start

In [34]:
df['POLYLINE'].isnull().sum()

0

There is no null value in this column. However, in order to see if there is inconsistent data in this column, I need to transfrom this column and extract coordinates into new columns. I will do this in the Feature Engineering step and do another round of Data Cleansing afterwards.

In [35]:
#Uploading current df to object store

In [36]:
df.sample(10)

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,POLYLINE
26584,1373092588620000142,C,NaN,NaN,20000142,1373092588,"[[-8.613639,41.147019],[-8.613621,41.146992],[..."
564008,1383126568620000159,B,NaN,15.0,20000159,1383126568,"[[-8.585694,41.148837],[-8.585685,41.148819],[..."
1082006,1392963878620000900,B,NaN,57.0,20000900,1392963878,"[[-8.610525,41.145174],[-8.610741,41.145201],[..."
687892,1385463188620000042,A,14133.0,NaN,20000042,1385463188,"[[-8.625096,41.158242],[-8.625078,41.15826],[-..."
796861,1387439140620000080,B,NaN,51.0,20000080,1387439140,"[[-8.621109,41.161014],[-8.621091,41.161023],[..."
326256,1379061842620000086,B,NaN,21.0,20000086,1379061842,"[[-8.628831,41.161005],[-8.628651,41.160879],[..."
506108,1382083858620000686,B,NaN,13.0,20000686,1382083858,"[[-8.627859,41.158008],[-8.62893,41.15889],[-8..."
303075,1378612738620000075,B,NaN,23.0,20000075,1378612738,"[[-8.61264,41.146056],[-8.612298,41.145957],[-..."
5184,1372749214620000076,C,NaN,NaN,20000076,1372749214,"[[-8.604666,41.169933],[-8.604666,41.169951],[..."
1441241,1399595560620000451,C,NaN,NaN,20000451,1399595560,"[[-8.603046,41.183838],[-8.603028,41.183847],[..."
